In [1]:
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = untar_data(URLs.ML_100k)

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
last_skywalker = np.array([0.98,0.9,-0.9])
user1 = np.array([0.9,0.8,-0.6])
casablanca = np.array([-0.99,-0.3,0.8])

In [5]:
(last_skywalker*user1).sum(axis=0)

2.1420000000000003

In [6]:
(user1*casablanca).sum()

-1.611

In [7]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


### DataLoaders

In [9]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [10]:
n_user = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_user,n_factors)
movie_factors = torch.randn(n_movies,n_factors)


In [11]:
one_hot_3 = one_hot(3,n_user).float()

In [12]:
user_factors.shape,one_hot_3.shape, n_user

(torch.Size([944, 5]), torch.Size([944]), 944)

In [13]:
user_factors.t()@ one_hot_3

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

In [14]:
user_factors[3]

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

## Creating our CF from scratch


In [15]:
class DotProduct(Module):
  def __init__(self,n_user,n_movies,n_factors):
    self.user_factors = Embedding(n_user, n_factors)
    self.movie_factors = Embedding(n_movies, n_factors)

  def forward(self,x):
    users = self.user_factors(x[:,0])
    movies = self.movie_factors(x[:,1])
    return (users*movies).sum(dim=1)

In [16]:
x,y = dls.one_batch()
x.shape,y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [17]:
model = DotProduct(n_user,n_movies,50)
Learn = Learner(dls,model,loss_func=MSELossFlat() )

In [18]:
Learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,1.344786,1.279100,00:11
1,1.093332,1.109981,00:11
2,0.958258,0.990199,00:11
3,0.814234,0.894916,00:11
4,0.780714,0.882022,00:11


In [19]:
class DotProduct(Module):
  def __init__(self,n_user,n_movies,n_factors,y_range=(0,5.5)):
    self.user_factors = Embedding(n_user, n_factors)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.y_range = y_range

  def forward(self,x):
    users = self.user_factors(x[:,0])
    movies = self.movie_factors(x[:,1])
    return sigmoid_range((users*movies).sum(dim=1),*self.y_range)

In [20]:
# this is run after sigmoid_range is added to the DotProduct classs
model = DotProduct(n_user,n_movies,50)
Learn = Learner(dls,model,loss_func=MSELossFlat())
Learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,0.986799,1.005294,00:11
1,0.878134,0.918898,00:11
2,0.675850,0.875467,00:13
3,0.483372,0.877939,00:11
4,0.378927,0.881887,00:11


In [21]:
class DotProduct(Module):
  def __init__(self,n_user,n_movies,n_factors,y_range=(0,5.5)):
    self.user_factors = Embedding(n_user, n_factors)
    self.user_bias = Embedding(n_user,1)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.movie_bias = Embedding(n_movies,1)
    self.y_range = y_range

  def forward(self,x):
    users = self.user_factors(x[:,0])
    movies = self.movie_factors(x[:,1])
    res = (users*movies).sum(dim=1, keepdim=True)
    res += (self.user_bias(x[:,0])) + self.movie_bias(x[:,1])
    return sigmoid_range(res,*self.y_range)

In [22]:
# this is run after bias is added to the DotProduct classs
model = DotProduct(n_user,n_movies,50)
Learn = Learner(dls,model,loss_func=MSELossFlat())
Learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,0.938634,0.952516,00:12
1,0.846664,0.865633,00:12
2,0.608090,0.865127,00:12
3,0.413482,0.887318,00:12
4,0.286971,0.894876,00:12


### Weight Decay 0r L2 Regularization

* loss_with_wd = loss + wd *(parameters**2).sum()
* parameters.grad + = wd * 2 * parameters

In [23]:
# this is run after wd (weight decay) is added to the DotProduct classs
model = DotProduct(n_user,n_movies,50)
Learn = Learner(dls,model,loss_func=MSELossFlat())
Learn.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.932776,0.961672,00:12
1,0.888625,0.882614,00:12
2,0.771066,0.832743,00:12
3,0.599807,0.822374,00:12
4,0.504981,0.822528,00:11


## Creating our own Module

In [24]:
class T(Module):
  def __init__(self): self.a = torch.ones(3)

L(T().parameters())

(#0) []

In [25]:
x= torch.ones(3);x

tensor([1., 1., 1.])

In [26]:
class T(Module):
  def __init__(self): self.a = nn.Parameter(torch.ones(3))

t=T()
L(t.parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [27]:
class T(Module):
  def __init__(self): self.a = nn.Linear(1,3,bias=False)

t=T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.3292],
        [-0.8623],
        [ 0.0592]], requires_grad=True)]

In [28]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [34]:
def creat_param(size):
  return nn.Parameter(torch.zeros(*size).normal_(0,0.01))

In [39]:
class DotProduct(Module):
  def __init__(self,n_user,n_movies,n_factors,y_range=(0,5.5)):
    self.user_factors = creat_param([n_user, n_factors])
    self.user_bias = creat_param([n_user])
    self.movie_factors = creat_param([n_movies, n_factors])
    self.movie_bias = creat_param([n_movies])
    self.y_range = y_range

  def forward(self,x):
    users = self.user_factors[x[:,0]]
    movies = self.movie_factors[x[:,1]]
    res = (users*movies).sum(dim=1)
    res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
    return sigmoid_range(res,*self.y_range)

In [40]:
model = DotProduct(n_user,n_movies,50)
Learn = Learner(dls,model,loss_func=MSELossFlat())
Learn.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.932825,0.947420,00:12
1,0.832693,0.874986,00:12
2,0.731426,0.834056,00:12
3,0.578805,0.820864,00:12
4,0.471141,0.821329,00:12
